In [45]:
import numpy as np
from scipy.spatial import distance
import base64

def hexdec(s):
    return bytes.fromhex(s)

def fixed_xor(b1, b2):
    a = np.frombuffer(b1, 'u1')
    b = np.frombuffer(b2, 'u1')
    return (a ^ b).tobytes()

ascii_hist = lambda s: np.histogram(s, bins=range(128), density=True)[0]

with open('t8.shakespeare.txt', 'r') as f:
    shks = np.frombuffer(f.read().encode("ASCII"), 'u1')

shks_hist = ascii_hist(shks)

score = lambda buf: distance.jensenshannon(shks_hist, ascii_hist(np.frombuffer(buf, 'u1')))

i2b = lambda i: bytes([i])

def break_sb_xor_key(cipher):
    evaluation = np.array([score(fixed_xor(cipher, i2b(i) * len(cipher))) for i in range(128)])
    return i2b(np.argmin(evaluation))

def rk_xor_key(plain, k):
    N = len(plain)
    n = len(k)
    return (k * np.ceil(N / n).astype(int))[:N]

def vigenere(plain, k):
    return fixed_xor(plain, rk_xor_key(plain, k))

In [5]:
hamming_distance = lambda x, y: int.from_bytes(fixed_xor(x, y), byteorder='big').bit_count()

In [6]:
hamming_distance(b'this is a test', b'wokka wokka!!!')

37

In [9]:
hamming_distance(hexdec('0000000000FFFF'), hexdec('FFFFFFFFFFFFFF'))

40

In [14]:
with open("6.txt", "rb") as f:
    cipher = base64.decodebytes(f.read())

In [17]:
def hamming_score(k_sz, cipher, n = 1):
    d = 0
    for i in range(n):
        d = d + hamming_distance(cipher[i*k_sz:(i+1)*k_sz], cipher[(i+1)*k_sz:(i+2)*k_sz]) / k_sz
        d /= 2
    return d

In [36]:
kz_sweep = np.array([hamming_score(kz, cipher, n=8) for kz in range(2, 41)])
kz_sorted = np.sort(kz_sweep)
candidate_scores = kz_sorted[:32]
candidates = [np.where(kz_sweep == candidate_scores[i])[0][0] + 2 for i in range(32)]

In [37]:
KEYSIZE = candidates[0]
cipherd = np.frombuffer(cipher, 'u1')
cipher_slices = np.pad(cipherd, (0, int(np.ceil(cipherd.shape[0] / KEYSIZE) * KEYSIZE - cipherd.shape[0])), mode='constant', constant_values=0).reshape(-1, KEYSIZE)
cipher_slices

array([[ 29,  66,  31, ...,   1,  22,  69],
       [ 54,   0,  30, ...,  12,  83,  18],
       [ 60,  12,  30, ...,   7,  83,   0],
       ...,
       [ 55,  73,  82, ...,  73,  28,  11],
       [120,  69,  49, ...,  73,  30,  16],
       [ 39,  12,  17, ...,   0,   0,   0]], dtype=uint8)

In [41]:
key_candidate = b''.join(break_sb_xor_key(cipher_slices.T[i].tobytes()) for i in range(KEYSIZE))
key_candidate

b'Terminator X: Bring the noise'

In [46]:
score(vigenere(cipher, key_candidate))

0.17412917534335998

In [53]:
def break_vigenere(cipher):
    kz_sweep = np.array([hamming_score(kz, cipher, n=8) for kz in range(2, 41)])
    kz_sorted = np.sort(kz_sweep)
    candidate_scores = kz_sorted[:8]
    candidates = [np.where(kz_sweep == candidate_scores[i])[0][0] + 2 for i in range(8)]

    key_candidates = []

    cipherd = np.frombuffer(cipher, 'u1')

    for KEYSIZE in candidates:
        cipher_slices = np.pad(cipherd, (0, int(np.ceil(cipherd.shape[0] / KEYSIZE) * KEYSIZE - cipherd.shape[0])), mode='constant', constant_values=0).reshape(-1, KEYSIZE)
        key_candidate = b''.join(break_sb_xor_key(cipher_slices.T[i].tobytes()) for i in range(KEYSIZE))
        key_candidates.append(key_candidate)
    
    scores = np.array([score(vigenere(cipher, key)) for key in key_candidates])

    return key_candidates[np.argmin(scores)]

In [57]:
print(vigenere(cipher, break_vigenere(cipher)).decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 
